In [ ]:
import random
import wandb
import torch
import torch.nn.functional as F
import torch_geometric.transforms as T 

from query_strategies import *
from augmentation import *
from model import *
from model_wrapper import *
from trainers import *
from util import *





c:\Users\banfi\anaconda3\envs\graph_active\Lib\site-packages\torch_geometric\typing.py:124: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: [WinError 127] A megadott eljárás nem található
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "


In [3]:
wandb.login()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: banfizsombor1999. Use `wandb login --relogin` to force relogin


True

In [4]:
random.seed(0)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
# dataset = Planetoid(root='/tmp/Cora', name='Cora')
# generate_balanced_data_splits(dataset,10,"data_splits\\cora_splits")

In [6]:
DROPOUT= 0.3
NUM_PASSES = 10
BUDGET = 150
EPOCHS = 100
SIGNIFICANCE_ITERATIONS = 10

NOISE_PROB = 0.4
NOISE_LEVEL = 0.5

In [7]:
# Augmentations
drop_edge = DropEdge(DROPOUT)
noise_feature_all = NoiseFeature(NOISE_LEVEL, 1)
noise_feature_col = NoiseFeature(NOISE_LEVEL, NOISE_PROB, "col")
noise_feature_row = NoiseFeature(NOISE_LEVEL, NOISE_PROB, "row")
noise_latent = NoiseLatent(NOISE_LEVEL)

drop_edge_noise_all = T.Compose([drop_edge, noise_feature_all])
drop_edge_noise_col = T.Compose([drop_edge, noise_feature_col])
drop_edge_noise_row = T.Compose([drop_edge, noise_feature_row])

In [9]:
# Strategies

random_query = RandomQuery()
entropy_query = EntropyQuery()

augment_sum_entropy = AugmentGraphSumEntropyQuery(drop_edge_noise_all, NUM_PASSES,0.0)
augment_logit_change = AugmentGraphLogitChange(drop_edge_noise_all, NUM_PASSES,1.0)
augment_latent = AugmentGraphSumQueryLatent(noise_latent, NUM_PASSES)
augment_sum_entropy_with_original = AugmentGraphSumEntropyQuery(drop_edge_noise_all, NUM_PASSES, 1/NUM_PASSES)


In [10]:
data_splits = [torch.load(f"data_splits\\cora_splits\\split_{i}.pt") for i in range(10)]


In [11]:
def init_wandb(query_strategy,run):
  config={
    "learning_rate": 0.01,
    "architecture": "GCN",
    "dataset": "CORA",
    "epochs": 200,
    "strategy": str(query_strategy)
    }
  
  if hasattr(query_strategy, "augmentation_fn"):
    augmentation = query_strategy.augmentation_fn
    config["augmentations"] = []
    if isinstance(augmentation, T.Compose):
      transforms = augmentation.transforms
    else:
      transforms = [augmentation]
      
    for t in transforms:
      aug_config = {}
      aug_config["name"] = str(t)
      aug_config["hyperparameters"] = t.__hyperparameters__()
      config["augmentations"].append(aug_config)
  
  wandb.init(
    # Set the project where this run will be logged
    project="graph-active-learning",
    # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
    name=f"{query_strategy}_{run}",
    # Track hyperparameters and run metadata
    config=config)
  return config

In [12]:
# config = {
#     "model": GCN_Contrastive(num_features, 7).to(device),
#     "loss_fn": nt_xent_loss,
#     "optimizer": torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4),
#     "trainer": ContrastiveTrainer(model, optimizer,loss_fn,drop_edge),
#     "strategy": EntropyQueryContrastive()
# }
# configs = [config]

In [14]:
STRATEGIES = [ augment_sum_entropy_with_original]
# STRATEGIES = noises_latent
final_accs = torch.zeros((len(STRATEGIES), BUDGET, SIGNIFICANCE_ITERATIONS))
final_auc = torch.zeros((len(STRATEGIES), BUDGET, SIGNIFICANCE_ITERATIONS))

# data_splits = [generate_random_data_split(dataset,10,500) for _ in range(SIGNIFICANCE_ITERATIONS)]
data_splits = [torch.load(f"data_splits\\cora_splits\\split_{i}.pt") for i in range(10)]

for strategy_ix, strategy in enumerate(STRATEGIES):
    print(f"Strategy: {strategy}")
    init_wandb(str(strategy), 4)
    
    
    
    for b in range(1,BUDGET+1):
        
        budget_accuracies = []
        budget_aucs = []
        
        
        
        for si in range(SIGNIFICANCE_ITERATIONS):
            dataset = data_splits[si].to(device)
            
            num_features = dataset.num_features
            num_classes = dataset.y.max().item() + 1
            
            print(f"{b} - {si} - {strategy}")
            model = GCN(num_features,num_classes).to(device)
            
            loss_fn = F.nll_loss
            optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
            wrapped_model = ModelWrapper(model,optimizer,loss_fn)
            trainer = Trainer()
            
            trainer.train(wrapped_model,dataset,200)
            acc = trainer.test(wrapped_model,dataset)
            
            budget_accuracies.append(acc)
            # budget_aucs.append(auc)
            
            query_node_idx = strategy(wrapped_model,dataset,dataset.train_pool)
            print(f'\tQuery node: {query_node_idx}')
            
            dataset.train_mask[query_node_idx] = True
            dataset.train_pool[query_node_idx] = False

        budget_accuracies = torch.tensor(budget_accuracies)
        budget_aucs = torch.tensor(budget_aucs)
        final_accs[strategy_ix, b-1, :] = budget_accuracies
        # final_auc[strategy_ix, b-1, :] = budget_aucs
        m = budget_accuracies.mean()
        std = budget_accuracies.std()
        wandb.log({"accuracy_mean": m.item(), "step":b})
        wandb.log({"accuracy_std": std.item(), "step": b})
wandb.finish()


Strategy: AugmentGraphSumEntropyQuery([DropEdge, NoiseFeature], original_weight=0.1)


1 - 0 - AugmentGraphSumEntropyQuery([DropEdge, NoiseFeature], original_weight=0.1)
	Query node: 1480
1 - 1 - AugmentGraphSumEntropyQuery([DropEdge, NoiseFeature], original_weight=0.1)
	Query node: 882
1 - 2 - AugmentGraphSumEntropyQuery([DropEdge, NoiseFeature], original_weight=0.1)
	Query node: 143
1 - 3 - AugmentGraphSumEntropyQuery([DropEdge, NoiseFeature], original_weight=0.1)
	Query node: 1241
1 - 4 - AugmentGraphSumEntropyQuery([DropEdge, NoiseFeature], original_weight=0.1)
	Query node: 1453
1 - 5 - AugmentGraphSumEntropyQuery([DropEdge, NoiseFeature], original_weight=0.1)
	Query node: 318
1 - 6 - AugmentGraphSumEntropyQuery([DropEdge, NoiseFeature], original_weight=0.1)
	Query node: 1127
1 - 7 - AugmentGraphSumEntropyQuery([DropEdge, NoiseFeature], original_weight=0.1)
	Query node: 1399
1 - 8 - AugmentGraphSumEntropyQuery([DropEdge, NoiseFeature], original_weight=0.1)
	Query node: 830
1 - 9 - AugmentGraphSumEntropyQuery([DropEdge, NoiseFeature], original_weight=0.1)
	Query node:

KeyboardInterrupt: 

In [15]:
wandb.finish()

accuracy_mean,▁▁▃▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇██▇▇▇▇█████████
accuracy_std,▅█▅▃▅▄▃▃▄▃▃▃▃▃▂▂▂▁▂▂▂▂▂▁▂▂▁▁▂▁▂▁▂▂▂▂▂▁▁▂
step,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▇▇▇▇▇██
accuracy_mean,0.7632
accuracy_std,0.01599
step,46
